In [1]:
from typing import *
import random

In [2]:
from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [3]:
from transformers import AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader

2022-12-14 01:00:19.102544: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
import sys
sys.path.append('..')

In [5]:
from model.par_with_attr import ParWithAttr
from model.base_par import DepParser
from utils import arc_rel_loss, uas_las, to_cuda

# Config

In [6]:
class CFG:
    # domains = ['BC', 'FIN', 'LEG', 'PB', 'PC', 'ZX']
    train_file = '../aug/diag_weakcodt_sampled_new/diag_train_merged.conll'
    dev_file = '../aug/codt/codt_dev.conll'
    domains = ['BC']
    # plm = 'hfl/chinese-electra-180g-large-discriminator'
    plm = 'hfl/chinese-electra-180g-base-discriminator'
    random_seed = 42
    # num_epochs = 15
    num_epochs = 15
    # batch_size = 64  # using 22g memory when using electra-large
    batch_size = 32
    plm_lr = 2e-5
    head_lr = 1e-4
    weight_decay = 0.01
    dropout = 0.2
    grad_clip = 2
    scheduler = 'linear'
    warmup_ratio = 0.1
    num_early_stop = 3
    max_length = 160
    hidden_size = 400
    num_labels = 35
    # print_every = 400
    # eval_every = 800
    print_every = 2000
    eval_every = 4000
    cuda = True
    fp16 = True

# Count

In [7]:
rel_dct = {
    'root': '根节点',
    'sasubj-obj': '同主同宾',
    'sasubj': '同主语',
    'dfsubj': '不同主语',
    'subj': '主语',
    'subj-in': '内部主语',
    'obj': '宾语',
    'pred': '谓语',
    'att': '定语',
    'adv': '状语',
    'cmp': '补语',
    'coo': '并列',
    'pobj': '介宾',
    'iobj': '间宾',
    'de': '的',
    'adjct': '附加',
    'app': '称呼',
    'exp': '解释',
    'punc': '标点',
    'frag': '片段',
    'repet': '重复',
    # rst
    'attr': '归属',
    'bckg': '背景',
    'cause': '因果',
    'comp': '比较',
    'cond': '状况',
    'cont': '对比',
    'elbr': '阐述',
    'enbm': '目的',
    'eval': '评价',
    'expl': '解释-例证',
    'joint': '联合',
    'manner': '方式',
    'rstm': '重申',
    'temp': '时序',
    # 'tp-chg': '主题变更',
    # 'prob-sol': '问题-解决',
    # 'qst-ans': '疑问-回答',
    # 'stm-rsp': '陈述-回应',
    # 'req-proc': '需求-处理',
}

In [8]:
rel2id = {}
for i, (key, value) in enumerate(rel_dct.items()):
    rel2id[key] = i
print(rel2id)
print(len(rel2id))

{'root': 0, 'sasubj-obj': 1, 'sasubj': 2, 'dfsubj': 3, 'subj': 4, 'subj-in': 5, 'obj': 6, 'pred': 7, 'att': 8, 'adv': 9, 'cmp': 10, 'coo': 11, 'pobj': 12, 'iobj': 13, 'de': 14, 'adjct': 15, 'app': 16, 'exp': 17, 'punc': 18, 'frag': 19, 'repet': 20, 'attr': 21, 'bckg': 22, 'cause': 23, 'comp': 24, 'cond': 25, 'cont': 26, 'elbr': 27, 'enbm': 28, 'eval': 29, 'expl': 30, 'joint': 31, 'manner': 32, 'rstm': 33, 'temp': 34}
35


# Seed & Device

In [9]:
def seed_everything(seed=CFG.random_seed):
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False

seed_everything()

In [10]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'Using device: {device}')

Using device: cuda


# Data

In [11]:
class Dependency():
    def __init__(self, idx, word, tag, head, rel):
        self.id = idx
        self.word = word
        self.tag = tag
        self.head = head
        self.rel = rel

    def __str__(self):
        # example:  1	上海	_	NR	NR	_	2	nn	_	_
        values = [str(self.idx), self.word, "_", self.tag, "_", "_", str(self.head), self.rel, "_", "_"]
        return '\t'.join(values)

    def __repr__(self):
        return f"({self.word}, {self.tag}, {self.head}, {self.rel})"

In [12]:
def load_codt(data_file: str):
    # id, form, tag, head, rel
#     sentence:List[Dependency] = [Dependency('0', '<root>', '_', '0', '_')]
    sentence:List[Dependency] = []

    with open(data_file, 'r', encoding='utf-8') as f:
        # data example: 1	上海	_	NR	NR	_	2	nn	_	_
        for line in f.readlines():
            toks = line.split()
            if len(toks) == 0:
                yield sentence
#                 sentence = [Dependency('0', '<root>', '_', '0', '_')]
                sentence = []
            elif len(toks) == 10:
                dep = Dependency(toks[0], toks[1], toks[3], toks[8], toks[9])
                sentence.append(dep)

from itertools import chain

def load_codt_with_aug(data_file: str, train: bool):
    # id, form, tag, head, rel
#     sentence:List[Dependency] = [Dependency('0', '<root>', '_', '0', '_')]
    sentence:List[Dependency] = []

    f1 = open(data_file, 'r', encoding='utf-8')
    f2 = open('../aug/codt/codt_train_fixed.conll', 'r', encoding='utf-8') if train else open('../aug/codt/codt_dev.conll', 'r', encoding='utf-8')
    for line in chain(f1.readlines(), f2.readlines()):
        toks = line.split()
        if len(toks) == 0:
            yield sentence
#                 sentence = [Dependency('0', '<root>', '_', '0', '_')]
            sentence = []
        elif len(toks) == 10:
            if toks[8] != '_':
                dep = Dependency(toks[0], toks[1], toks[3], toks[8], toks[9])
            else:
                dep = Dependency(toks[0], toks[1], toks[3], toks[6], toks[7])
            sentence.append(dep)

    f1.close()
    f2.close()

In [13]:
tokenizer = AutoTokenizer.from_pretrained(CFG.plm)
CFG.tokenizer = tokenizer

In [16]:
class CODTDataset(Dataset):
    def __init__(self, cfg, train):
        self.train = train
        self.cfg = cfg
        self.inputs, self.offsets, self.heads, self.rels, self.masks = self.read_data()
        
    def read_data(self):
        inputs, offsets = [], []
        tags, heads, rels, masks = [], [], [], []
        
        for domain in self.cfg.domains:
            file = self.cfg.train_file if self.train else self.cfg.dev_file
            
            load_fn = load_codt_with_aug
            for deps in tqdm(load_fn(file, self.train)):
            # for deps in tqdm(load_codt(file)):
                # another sentence
                seq_len = len(deps)
                
                word_lst = [] 
                rel_attr = {'input_ids':torch.Tensor(), 'token_type_ids':torch.Tensor(), 'attention_mask':torch.Tensor()}
                head_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)  # same as root index is 0, constrainting by mask 
                rel_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)
                mask_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)
                for i, dep in enumerate(deps):
                    if i == seq_len or i + 1== self.cfg.max_length:
                        break
                        
                    word_lst.append(dep.word)
                      
                    if dep.head in ['_', '-1'] or int(dep.head) + 1 >= self.cfg.max_length:
                        head_tokens[i+1] = 0
                        mask_tokens[i+1] = 0
                    else:
                        head_tokens[i+1] = int(dep.head)
                        mask_tokens[i+1] = 1

                    if self.train:
                        rel_tokens[i+1] = rel2id[dep.rel]
                    else:
                        rel_tokens[i+1] = rel2id.get(dep.rel, rel2id['adjct'])

                tokenized = tokenizer.encode_plus(word_lst, 
                                                  padding='max_length', 
                                                  truncation=True,
                                                  max_length=self.cfg.max_length, 
                                                  return_offsets_mapping=True, 
                                                  return_tensors='pt',
                                                  is_split_into_words=True)
                inputs.append({"input_ids": tokenized['input_ids'][0],
                              "token_type_ids": tokenized['token_type_ids'][0],
                               "attention_mask": tokenized['attention_mask'][0]
                              })
                
#                 sentence_word_idx = np.zeros(self.cfg.max_length, dtype=np.int64)
                sentence_word_idx = []
                for idx, (start, end) in enumerate(tokenized.offset_mapping[0][1:]):
                    if start == 0 and end != 0:
                        sentence_word_idx.append(idx)
#                         sentence_word_idx[idx] = idx
                if len(sentence_word_idx) < self.cfg.max_length - 1:
                    sentence_word_idx.extend([0]* (self.cfg.max_length - 1 - len(sentence_word_idx)))
                offsets.append(torch.as_tensor(sentence_word_idx))
                
                heads.append(head_tokens)
                rels.append(rel_tokens)
                masks.append(mask_tokens)
                    
        return inputs, offsets, heads, rels, masks

    def __getitem__(self, idx):
        return self.inputs[idx], self.offsets[idx], self.heads[idx], self.rels[idx], self.masks[idx]
    
    def __len__(self):
        return len(self.rels)

In [17]:
train_dataset = CODTDataset(CFG, train=True)
val_dataset = CODTDataset(CFG, train=False)

59458it [01:52, 528.27it/s]
1948it [00:03, 521.28it/s]


In [18]:
train_iter = DataLoader(train_dataset, batch_size=CFG.batch_size)
val_iter = DataLoader(val_dataset, batch_size=CFG.batch_size)

# Model

In [19]:
model = DepParser(CFG)

Some weights of the model checkpoint at hfl/chinese-electra-180g-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Trainer

In [20]:
class MyTrainer():
    def __init__(self, 
                 model,
                 trainset_size,
                 loss_fn: Callable, 
                 metrics_fn: Callable, 
                 config: Dict) -> None:
        self.loss_fn = loss_fn
        self.metrics_fn = metrics_fn

        plm_params = [p for n,p in model.named_parameters() if 'encoder' in n]
        head_params = [p for n,p in model.named_parameters() if 'encoder' not in n]
        self.optim = AdamW([{'params': plm_params, 'lr':config.plm_lr}, 
                            {'params': head_params, 'lr':config.head_lr}], 
                            lr=config.plm_lr,
                            weight_decay=config.weight_decay
                          )
        
        training_step = int(config.num_epochs * (trainset_size / config.batch_size))
        warmup_step = int(config.warmup_ratio * training_step)  
        self.optim_schedule = get_linear_schedule_with_warmup(optimizer=self.optim, 
                                                              num_warmup_steps=warmup_step, 
                                                              num_training_steps=training_step)
        self.scaler = torch.cuda.amp.GradScaler(enabled=config.fp16)

        self.config = config

    def train(self, 
              model: nn.Module, 
              train_iter: DataLoader, 
              val_iter: DataLoader):
        model.train()
        if self.config.cuda and torch.cuda.is_available():
            model.cuda()
            pass
        
        best_res = [0, 0, 0]
        early_stop_cnt = 0
        best_state_dict = None
        step = 0
        for epoch in tqdm(range(self.config.num_epochs)):
            for batch in train_iter:
                inputs, offsets, heads, rels, masks = batch
                
                if self.config.cuda and torch.cuda.is_available():
                    inputs_cuda = {}
                    for key, value in inputs.items():
                        inputs_cuda[key] = value.cuda()
                    inputs = inputs_cuda
                    
                    offsets, heads, rels, masks = to_cuda(data=(offsets, heads, rels, masks))
                
                arc_logits, rel_logits, loss = model(inputs, offsets, heads, rels, masks)
                
                self.optim.zero_grad()
                if self.config.cuda and self.config.fp16:
                    self.scaler.scale(loss).backward()
                    self.scaler.unscale_(self.optim)
                else:
                    loss.backward()

                nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), max_norm=self.config.grad_clip)

                if self.config.fp16:
                    self.scaler.step(self.optim)
                    self.scaler.update()
                else:
                    self.optim.step()
                self.optim_schedule.step()

                metrics = self.metrics_fn(arc_logits, rel_logits, heads, rels, masks)

                if (step) % self.config.print_every == 0:
                    print(f"--epoch {epoch}, step {step}, loss {loss}")
                    print(f"  {metrics}")

                if val_iter and (step + 1) % self.config.eval_every == 0:
                    avg_loss, uas, las = self.eval(model, val_iter)
                    res = [avg_loss, uas, las]
                    if las > best_res[2]:  # las
                        best_res = res
                        best_state_dict = model.state_dict()
                        torch.save(best_state_dict, f"../results/base_par_new.pt")
                        early_stop_cnt = 0
                    else:
                        early_stop_cnt += 1
                    
                    print("--Best Evaluation: ")
                    print("-loss: {}  UAS: {}  LAS: {} \n".format(*best_res))
                    # back to train mode
                    model.train()
                
                if early_stop_cnt >= self.config.num_early_stop:
                    print("--early stopping, training finished.")
                    return best_res, best_state_dict

                step += 1
        print("--training finished.")
        if best_state_dict is not None:
            return best_res, best_state_dict
        return best_res, model.state_dict()

    # eval func
    def eval(self, model: nn.Module, eval_iter: DataLoader, save_file: str = "", save_title: str = ""):
        model.eval()

        head_whole, rel_whole, mask_whole = torch.Tensor(), torch.Tensor(), torch.Tensor()
        arc_logit_whole, rel_logit_whole = torch.Tensor(), torch.Tensor()
        avg_loss = 0.0
        for step, batch in enumerate(eval_iter):
            inputs, offsets, heads, rels, masks = batch

            if self.config.cuda and torch.cuda.is_available():
                inputs_cuda = {}
                for key, value in inputs.items():
                    inputs_cuda[key] = value.cuda()
                inputs = inputs_cuda

                offsets, heads, rels, masks = to_cuda(data=(offsets, heads, rels, masks))
            
            with torch.no_grad():
                arc_logits, rel_logits = model(inputs, offsets, heads=None, rels=None, masks=masks, evaluate=True)
            
            loss = arc_rel_loss(arc_logits, rel_logits, heads, rels, masks)

            arc_logit_whole = torch.cat([arc_logit_whole, arc_logits.cpu()], dim=0)
            rel_logit_whole = torch.cat([rel_logit_whole, rel_logits.cpu()], dim=0)

            head_whole, rel_whole = torch.cat([head_whole, heads.cpu()], dim=0), torch.cat([rel_whole, rels.cpu()], dim=0)
            mask_whole = torch.cat([mask_whole, masks.cpu()], dim=0)

            avg_loss += loss.item() * len(heads)  # times the batch size of data

        metrics = self.metrics_fn(arc_logit_whole, rel_logit_whole, head_whole, rel_whole, mask_whole)
        uas, las = metrics['UAS'], metrics['LAS']

        avg_loss /= len(eval_iter.dataset)  # type: ignore

        print("--Evaluation:")
        print("Avg Loss: {}  UAS: {}  LAS: {} \n".format(avg_loss, uas, las))

        if save_file != "":
            results = [save_title, avg_loss, uas, las]  # type: ignore
            results = [str(x) for x in results]
            with open(save_file, "a+") as f:
                f.write(",".join(results) + "\n")  # type: ignore

        return avg_loss, uas, las  # type: ignore
    
    def save_results(self, save_file, save_title, results):
        saves = [save_title] + results
        saves = [str(x) for x in saves]
        with open(save_file, "a+") as f:
            f.write(",".join(saves) + "\n")  # type: ignore

# Training

In [21]:
trainer = MyTrainer(model=model, trainset_size=len(train_dataset), loss_fn=arc_rel_loss, metrics_fn=uas_las, config=CFG)
best_res, best_state_dict = trainer.train(model=model, train_iter=train_iter, val_iter=None)

/home/jgy/.conda/envs/jgy/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/15 [00:00<?, ?it/s]

--epoch 0, step 0, loss 9.239877700805664
  {'UAS': 0.01607717041800643, 'LAS': 0.0}


  7%|▋         | 1/15 [11:00<2:34:05, 660.39s/it]

--epoch 1, step 2000, loss 0.1817149519920349
  {'UAS': 0.9422222222222222, 'LAS': 0.9333333333333333}


In [ ]:
trainer.save_results(save_file='../results/res.txt', save_title='', results=best_res)

In [ ]:
torch.save(best_state_dict, f"../results/base_par_new.pt")

In [ ]:
model = None
torch.cuda.empty_cache()